# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
print(len(song_files))

79


In [6]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json


In [30]:
df = pd.read_json(filepath, lines = True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [12]:
#song_data = df.loc[:,['song_id','title','artist_id','year','duration']].values[0].tolist()
#song_data

array([['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]], dtype=object)

In [25]:
song_data = df[['song_id','title','artist_id','year','duration']].drop_duplicates()
#song_data = song_data.drop_duplicates()
song_data = song_data.replace({np.nan: None})
song_data  = song_data.values[0].tolist()
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
song_table_insert

'\nINSERT INTO songs(song_id, title, artist_id,year, duration)\nVALUES (%s, %s, %s, %s, %s) \non conflict do nothing;\n'

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

ProgrammingError: relation "songs" does not exist
LINE 2: INSERT INTO songs(song_id, title, artist_id,year, duration)
                    ^


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df.loc[:,["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values[0].tolist()
artist_data

['ARXR32B1187FB57099', 'Gob', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
artist_table_insert

'\nINSERT INTO artists (artist_id, name, location, latitude, longitude)\nVALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING;\n'

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [5]:
log_files = get_files("data/log_data")

In [6]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-29-events.json


In [7]:
df = pd.read_json(filepath, lines = True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [17]:
df = df.query("page == 'NextSong'")
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Fu,Logged In,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540007e+12,514,Ja I Ty,200,1542069637796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
3,All Time Low,Logged In,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540677e+12,510,A Party Song (The Walk of Shame),200,1542071549796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",51
4,Nik & Jay,Logged In,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",PUT,NextSong,1.540872e+12,379,Pop-Pop!,200,1542079142796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....,9
5,Quad City DJ's,Logged In,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,506,C'mon N' Ride It (The Train) (LP Version),200,1542081112796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8,Hoobastank,Logged In,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",PUT,NextSong,1.540971e+12,492,Born To Lead,200,1542085206796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,94


In [18]:
x = df.head()


In [19]:
"""
for index , row in x.iterrows():
    print(index, row)
"""

'\nfor index , row in x.iterrows():\n    print(index, row)\n'

In [20]:
t = pd.to_datetime(df.loc[:,'ts'], unit='ms')
df['ts'] = pd.to_datetime(df.loc[:,'ts'], unit='ms') 

In [21]:
df['ts'].head()

1   2018-11-13 00:40:37.796
3   2018-11-13 01:12:29.796
4   2018-11-13 03:19:02.796
5   2018-11-13 03:51:52.796
8   2018-11-13 05:00:06.796
Name: ts, dtype: datetime64[ns]

In [22]:
time_data = list((t, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = list(('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'))

In [23]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
1,2018-11-13 00:40:37.796,0,13,46,11,2018,1
3,2018-11-13 01:12:29.796,1,13,46,11,2018,1
4,2018-11-13 03:19:02.796,3,13,46,11,2018,1
5,2018-11-13 03:51:52.796,3,13,46,11,2018,1
8,2018-11-13 05:00:06.796,5,13,46,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
user_df = df.loc[:,["userId", "firstName", "lastName", "gender", "level"]]

In [26]:
user_df.head()

,userId,firstName,lastName,gender,level
1,66,Kevin,Arellano,M,free
3,51,Maia,Burke,F,free
4,9,Wyatt,Scott,M,free
5,49,Chloe,Cuevas,F,free
8,94,Noah,Chavez,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [27]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId,row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [37]:
    # filter by NextSong action
    df = df.query("page == 'NextSong'")

    # parsed the ts column using to_datetime and convert it into a pandas dataframe
    t_s = pd.DataFrame({
        'StartTime': pd.to_datetime(df['ts'], unit='ms')
    })
    
    # New Columns
    t_s['hour'], t_s['day'], t_s['week'], t_s['month'], t_s['year'], t_s['weekday'] = t_s['StartTime'].dt.hour,\
    t_s['StartTime'].dt.day,t_s['StartTime'].dt.weekofyear, t_s['StartTime'].dt.month, t_s['StartTime'].dt.year, t_s['StartTime'].dt.weekday
    
    t_s = t_s.drop_duplicates()

In [40]:
 # load user table
user_df = df[["userId", "firstName", "lastName", "gender", "level"]].drop_duplicates()
user_df = user_df.replace({np.nan:None})
user_df.head(5)

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
21,50,Ava,Robinson,F,free
35,54,Kaleb,Cook,M,free
40,32,Lily,Burns,F,free
